In [123]:
import pandas as pd 
from datetime import datetime, date, timedelta
from itertools import combinations


In [225]:
DATE_FORMAT = "%Y-%m-%d"
TODAY_DT = date.today().strftime(DATE_FORMAT)


# 1. Loading the csv of dates

In [150]:
events_df = pd.read_csv("events.csv")
events_df.head()

,event,date,importance,category
0,Philip,1921-06-10,1,birth
1,Elizabeth,1926-04-21,1,birth
2,Engagement,1947-07-09,3,both
3,Wedding,1947-11-20,2,both
4,Charles' birth,1948-11-14,2,child


In [151]:
events = events_df.set_index('event').transpose().to_dict()
events

{'Philip': {'date': '1921-06-10', 'importance': 1, 'category': 'birth'},
 'Elizabeth': {'date': '1926-04-21', 'importance': 1, 'category': 'birth'},
 'Engagement': {'date': '1947-07-09', 'importance': 3, 'category': 'both'},
 'Wedding': {'date': '1947-11-20', 'importance': 2, 'category': 'both'},
 "Charles' birth": {'date': '1948-11-14',
  'importance': 2,
  'category': 'child'},
 "Anne's birth": {'date': '1950-08-15', 'importance': 2, 'category': 'child'},
 'Beginning of reign': {'date': '1952-02-06',
  'importance': 1,
  'category': 'both'},
 'Coronation': {'date': '1953-06-02', 'importance': 2, 'category': 'both'},
 "Andrew's birth": {'date': '1960-02-19',
  'importance': 2,
  'category': 'child'},
 "Edward's birth": {'date': '1964-03-10',
  'importance': 2,
  'category': 'child'},
 "Philip's death": {'date': '2021-04-09',
  'importance': 2,
  'category': 'death'},
 "Elizabeth's death": {'date': '2022-09-08',
  'importance': 1,
  'category': 'death'}}

# 2. Functions

In [222]:
def days_between(dt1, dt2):
    """Calculates difference between `dt1` and `dt2`.
    
    Args:
        dt1 (str like 'YYYY-MM-DD'): 
        dt2 (str like 'YYYY-MM-DD'): 
        
    Returns:
        Boolean
    """
    try:
        return abs((datetime.strptime(dt2, "%Y-%m-%d") - datetime.strptime(dt1, "%Y-%m-%d")).days)
    except:
        return None


In [227]:
# iterator by date
# date format of input args: date(YYYY, MM, DD)
# output format: 

def daterange(start_dt, end_dt):
    '''
    - start_dt - date(YYYY, MM, DD)
    - end_dt - date(YYYY, MM, DD)
    Output:
    - iterator of date(YYYY, MM, DD) from start_dt (including) to end_dt (not including)
    '''
    for n in range(int((end_dt - start_dt).days)):
        yield start_dt + timedelta(n)


In [236]:
# rules for detecting anniversaries

def rule_multiple(dt1, dt2, n):
    """Identifies if difference between 2 dates `dt1` and `dt2` is multiple of `n`.
    
    Args:
        dt1 (date(YYYY, MM, DD)): 
        dt2 (date(YYYY, MM, DD)): 
        n (int): The number of multiplicity.
        
    Returns:
        Boolean
    """
    if days_between(dt1.strftime('%Y-%m-%d'), dt2.strftime('%Y-%m-%d')) % n == 0:
        return True
    else:
        return False

def rule_anniversary(dt1, dt2):
    """Identifies if `dt1` is an anniversary of `dt2` or vice versa.
    
    Args:
        dt1 (date(YYYY, MM, DD)): 
        dt2 (date(YYYY, MM, DD)): 
        
    Returns:
        Int or None
    """
    if dt1.month == dt2.month and dt1.day == dt2.day:
        return abs(dt1.year - dt2.year)
    else:
        return None


In [133]:
def today_counter(events):
    print("Today is...")
    for event in events.keys():
        print("{} days since {}".format(days_between(TODAY_DT, events[event]['date']), event))

def range_calendar(dt_start, dt_end):
    for dt in daterange(dt_start, dt_end):
        for event in events.keys():
            event_dt = datetime.strptime(events[event]['date'], DATE_FORMAT)
            if (
                (rule_multiple(dt, event_dt, 100) and events[event]['importance'] <= 2)
                or
                (rule_multiple(dt, event_dt, 1000) and events[event]['importance'] == 3)
            ):
                print("{} -- {} days since {}".format(dt, days_between(dt.strftime(DATE_FORMAT), events[event]['date']), event))
            if rule_anniversary(dt, datetime.strptime(events[event]['date'], DATE_FORMAT)):
                print("{} -- {} years since {}".format(dt, rule_anniversary(dt, event_dt), event))
            


In [131]:

def differences_inside_set(events):
    for event1 in events.keys():
        for event2 in events.keys():
            if days_between(events[event1]['date'], events[event2]['date']) % 100 == 0 and event1 != event2 and events[event1]['date'] < events[event2]['date']:
                print("{} days between {} and {}".format(days_between(events[event1]['date'], events[event2]['date']), event1, event2))


In [228]:
def birth_dates(events):
    birth_dict = {}
    for k, v in events.items():
        if v['category'] == 'birth':
            birth_dict[k] = v['date']    
    return birth_dict

In [208]:
def age_counter(given_dt):
    
    birth_dict = birth_dates(events)
    output_dict = {}
    
    try:
        for k in birth_dict.keys():
#             print("{0}'s age: {1} days".format(k, days_between(given_dt, birth_dict[k])))
            output_dict["{0}'s age".format(k)] = days_between(given_dt, birth_dict[k])

        if len(birth_dict.keys()) > 1:
            for (k1, k2) in combinations(birth_dict.keys(), 2):
#                 print("Age difference between {0} and {1}: {2} days".format(k1, k2, days_between(birth_dict[k1], birth_dict[k2])))
                output_dict["Age difference between {0} and {1}".format(k1, k2)] = days_between(birth_dict[k1], birth_dict[k2])
#             print("Total age: {} days".format(sum([days_between(given_dt, birth_dict[k]) for k in birth_dict.keys()])))
            output_dict["Total age"] = sum([days_between(given_dt, birth_dict[k]) for k in birth_dict.keys()])
        print("Ages were calculated.")
    
    except:
        print("Something wrong happened...")

    if len(output_dict):
        return pd.DataFrame.from_dict(output_dict, orient='index', columns=['days'])
    else:
        return "Sorry, there is no answer :("
    

In [233]:
def total_age_anniversaries(start_dt, end_dt, n):
    """Returns dates between `start_dt` and `end_dt` in which total age (in days) 
    is close to be multiple of `n`.
    
    Args:
        start_dt (str like 'YYYY-MM-DD'): 
        end_dt (str like 'YYYY-MM-DD'): 
        n (int): The number for checking multiplicity (more or equal 10)
    
    Returns:
        DataFrame
    """
    birth_dict = birth_dates(events)
    output_dict = {}
    
    try:
        for dt in daterange(datetime.strptime(start_dt, "%Y-%m-%d"), datetime.strptime(end_dt, "%Y-%m-%d")):
            total_age = sum([days_between(dt.strftime('%Y-%m-%d'), birth_dict[k]) for k in birth_dict.keys()])
            if n >= 10 and total_age % n in range(len(birth_dict)):
#                 print("{} -- {} days of their combined age".format(dt.strftime('%Y-%m-%d'), total_age))
                output_dict[dt.strftime('%Y-%m-%d')] = total_age
    except:
        print("Something wrong happened...")
    
    if len(output_dict):
        return pd.DataFrame.from_dict(output_dict, orient='index', columns=['days'])
    else:
        return "Sorry, there is no answer :("
    

# 3. Performing

In [211]:
# age_counter(given_dt)
# age_counter("")

In [235]:
# total_age_anniversaries(events['Engagement']['date'], events["Philip's death"]['date'], 1000)

In [161]:
# today_counter(events)

In [160]:
# range_calendar(date.today(), date(2023, 12, 31))


In [129]:
# birthday_dt_1 = events["Elizabeth"]['date']
# birthday_dt_2 = events["Philip"]['date']
# given_dt = events["Philip's death"]['date']


In [157]:
# total_age_anniversaries(birthday_dt_1, birthday_dt_2, events['Engagement']['date'], events["Philip's death"]['date'])


In [156]:
# differences_inside_set(events)

In [212]:
if __name__=="__main__":
    print(age_counter('2021-04-09'))

Ages were calculated.
                                              days
Philip's age                                 36463
Elizabeth's age                              34687
Age difference between Philip and Elizabeth   1776
Total age                                    71150
